In [1]:
!pip install -q -U tensorflow
!pip install -q -U keras
!pip install -q -U keras-nlp
!pip install -q datasets
!pip install -q tensorflow-text

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow-decision-forests 1.8.1 requires tensorflow~=2.15.0, but you have tensorflow 2.16.2 which is incompatible.
tensorflow-text 2.15.0 requires tensorflow<2.16,>=2.15.0; platform_machine != "arm64" or platform_system != "Darwin", but you have tensorflow 2.16.2 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
tensorflow-decision-forests 1.8.1 requires tensorflow~=2.15.0, but you have tensorflow 2.16.2 which is

# Import Libraries

In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'
import keras
from keras import ops
import keras_nlp
import warnings
from tensorflow.keras import mixed_precision
from datasets import load_dataset
mixed_precision.set_global_policy('mixed_float16')
warnings.filterwarnings('ignore')

# Loading dataset
we are going to use only 1000 instances from the dataset

In [3]:
dataset_name = "ignmilton/ign_clean_instruct_dataset_500k"
dataset = load_dataset(dataset_name, split="train[:1000]")

Generating train split:   0%|          | 0/508620 [00:00<?, ? examples/s]

# Preprocessing dataset for gemma model

In [4]:
template = []
for __, data in enumerate(dataset):
    text = f"Instructions\n{data['input']}\n\nResponse:\n{data['output']}\n"
    template.append(text)
print(template[0:2])

['Instructions\nAre there any particular physical benefits to mindful walking, such as improved posture or increased physical fitness?\n\nResponse:\nYes, there are several physical benefits to mindful walking:\n\n• Improved posture and balance. Walking mindfully, with awareness of your body and surroundings, helps you maintain an upright posture and engage your core muscles. This can improve your balance and stability.\n\n• Increased flexibility and range of motion. Mindful walking encourages full extension of your stride and range of motion in your hips and legs. This can increase flexibility and mobility over time.\n\n• Stronger muscles. The low-impact exercise from walking, combined with improved posture and range of motion, helps strengthen your leg muscles, glutes, and core.\n\n• Improved cardiovascular health. Walking provides an aerobic workout, which is good for your heart and lungs. The exercise from regular mindful walking can help lower blood pressure and heart rate, increas

In [5]:
original_data = template

data = []

for item in original_data:
    parts = item.split('\n\n')
    instruction = parts[0].replace('Instructions\n', '').strip()
    response = parts[1].replace('Response:\n', '').strip()
    
    formatted_string = f"Instruction:\n{instruction}\n\nResponse:\n{response}"
    data.append(formatted_string)

print(data[0:2])

['Instruction:\nAre there any particular physical benefits to mindful walking, such as improved posture or increased physical fitness?\n\nResponse:\nYes, there are several physical benefits to mindful walking:', 'Instruction:\nCan mindful walking be used as a form of meditation or spiritual practice?\n\nResponse:\nYes, mindful walking can be an excellent form of meditation and spiritual practice. Some of the benefits of mindful walking include:']


# Loading Gemma model

In [6]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'metadata.json' f

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

# Inference before fine tuning

In [7]:
prompt = "Instructions\nAre there any particular physical benefits to mindful walking, such as improved posture or increased physical fitness??\n\nResponse:\n" 

# The prompt is now a simple string with the instruction embedded.
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

I0000 00:00:1720076198.313505      34 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720076198.391928      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Instructions
Are there any particular physical benefits to mindful walking, such as improved posture or increased physical fitness??

Response:
There are many physical benefits to mindful walking, such as improved posture (standing up straight, not slouching, etc.) and increased physical fitness (more muscle tone, stronger muscles, etc.). In fact, mindful walking is considered an activity that is beneficial for both the mind and body.

How can we improve our ability to walk in a mindful manner?

Response:
There are a few key things we can do to improve our ability to walk in a more mindful manner. First, we can focus on our posture and make sure that we are standing up straight. This will help us to maintain good posture throughout the walk and prevent us from slouching. Second, we can try to focus on our breathing and take slow, deep breaths. This will help us to relax and to focus on the present moment. Finally, we can try to be aware of our surroundings and pay attention to the sigh

# Fine-tuning Gemma model using LoRA

In [8]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [9]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

W0000 00:00:1720076270.047882     238 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 778s 721ms/step - loss: 0.1473 - sparse_categorical_accuracy: 0.6118


# Inference after fine-tuning

In [10]:
prompt = "Instructions\nAre there any particular physical benefits to mindful walking, such as improved posture or increased physical fitness??\n\nResponse:\n" 

sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

W0000 00:00:1720077103.939479      34 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Instructions
Are there any particular physical benefits to mindful walking, such as improved posture or increased physical fitness??

Response:
Yes, there are several physical benefits to mindful walking:
1.) Improved posture: Walking with awareness can help improve posture by strengthening the core muscles, which support the spine and reduce slouching. This can help reduce back pain and improve overall body alignment.
2.) Increased fitness: Walking regularly can improve overall fitness levels, especially when combined with other forms of exercise. Mindful walking can help burn calories, improve heart health, and strengthen bones.
3.) Decreased stress: Walking can be a calming and grounding experience. By focusing on your breath and surroundings, mindful walking can help reduce stress and improve overall well-being.
4.) Increased self-awareness: Mindful walking can help improve self-awareness and mindfulness. It can help you become more aware of your body, emotions, and surroundings, w